# Attempt 1

This notebook is my first attempt at the problem. I have tried to use a vision transformer as the model of choice. Since it is not using any pretrained weights, I have decided to use a contrastive pretraining task, where all images having the same printer are positive pairs, and images of different printers are negative pairs. A second contrastive pretraining was also applied, where images of prints having underextrusion or not having underextrusion are the positive pairs, and the opposite labelled images form the negative pairs. This model is finetuned for the classification task. 

In [ ]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import lightning.pytorch as pl
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models.vision_transformer import VisionTransformer
import seaborn as sns
from sklearn.model_selection import train_test_split
from tqdm import tqdm 
from PIL import Image
from lightning.pytorch.callbacks import RichProgressBar
from lightning.pytorch.callbacks import TQDMProgressBar

from lightning.pytorch.callbacks.progress.rich_progress import RichProgressBarTheme

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
dataframe = pd.read_csv("train.csv")

In [4]:
train_df, val_df = train_test_split(dataframe, test_size = 0.2, random_state = 0)

In [5]:
# Build SSCL dataset
total_size = 10000
sscl_dataset = pd.DataFrame(columns = ['simimage1', 'simimage2', 'image3'])

for printer in train_df['printer_id'].unique():
    df = (
        train_df[(train_df['printer_id'] == printer) & (train_df['has_under_extrusion'] == 0)], 
        train_df[(train_df['printer_id'] == printer) & (train_df['has_under_extrusion'] == 1)]
    )
    for idx in tqdm(range(total_size)):
        label_choice = np.random.randint(2)
        dissim_label = 1 - label_choice
        sim = df[label_choice].sample(2)
        dissim = df[dissim_label].sample(1)
        
        
        sscl_dataset.loc[len(sscl_dataset.index)] = [sim.iloc[0]['img_path'], sim.iloc[1]['img_path'], dissim.iloc[0]['img_path']] 
        
        

100%|██████████| 10000/10000 [00:56<00:00, 176.61it/s]


In [6]:
# Build SSCL dataset
total_size = 50000
# sscl_dataset = pd.DataFrame(columns = ['simimage1', 'simimage2', 'image3'])

df = (
    train_df[train_df['has_under_extrusion'] == 0],
    train_df[train_df['has_under_extrusion'] == 1]
)

for idx in tqdm(range(total_size)):

    label_choice = np.random.randint(2)
    dissim_label = 1 - label_choice
    sim = df[label_choice].sample(2)
    dissim = df[dissim_label].sample(1)


    sscl_dataset.loc[len(sscl_dataset.index)] = [sim.iloc[0]['img_path'], sim.iloc[1]['img_path'], dissim.iloc[0]['img_path']] 



100%|██████████| 50000/50000 [11:14<00:00, 74.13it/s] 


In [31]:
class SSCLDataset(Dataset):

    def __init__(self, dataset):
        # Transforms
        self.dataset = dataset
        self.to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize(size = (128, 128))
        self.grayscale = transforms.Grayscale()
        self.root_dir = "images/"
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        
        row = self.dataset.iloc[idx]
        anchor = self.resize(self.to_tensor(Image.open(self.root_dir + row['simimage1'])))
        positive = self.resize(self.to_tensor(Image.open(self.root_dir + row['simimage2'])))
        negative = self.resize(self.to_tensor(Image.open(self.root_dir + row['image3'])))
        return self.grayscale(anchor), self.grayscale(positive), self.grayscale(negative)
#         return anchor, positive, negative

In [32]:
sscl_dataloader = DataLoader(
    SSCLDataset(sscl_dataset),
    batch_size=16,
    shuffle=True,
    pin_memory=True,
    num_workers=36
)

In [33]:
class ViT(VisionTransformer):
    def __init__(self,image_size = 128, patch_size = 16, num_layers = 8, num_heads = 8, hidden_dim = 256, mlp_dim = 512):
        super().__init__(image_size, patch_size, num_layers, num_heads, hidden_dim, mlp_dim)
        
        self.n_patches = self.image_size // self.patch_size
        
        self.conv_proj = nn.Conv2d(1, hidden_dim, kernel_size=(patch_size, patch_size), stride=(patch_size, patch_size))
        #self.heads = None
        
    def forward(self, x: torch.Tensor):
        # Reshape and permute the input tensor
        x = self._process_input(x)
        n = x.shape[0]

        # Expand the class token to the full batch
        batch_class_token = self.class_token.expand(n, -1, -1)
        x = torch.cat([batch_class_token, x], dim=1)
        x = self.encoder(x)
        x = x[:, 0]
        
        
        
        return self.heads(x)

In [34]:
class ContrastiveModel(pl.LightningModule):
    
    def __init__(self, ):
        
        super().__init__()
        self.vit = ViT()
        self.criterion = nn.TripletMarginLoss()
        
    def forward(self, x):
        return self.vit(x)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.00003)
#         scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 1)
        
#         return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "train_loss"}
        return [optimizer]
    
    
    def training_step(self, batch, batch_idx):
        anchor, positive, negative = batch
        
        loss = self.criterion(self(anchor), self(positive), self(negative))
        self.log("train_loss", loss.item(), prog_bar=True)
        return loss
        

In [35]:
contrastive_model = ContrastiveModel()

In [36]:
progress_bar = RichProgressBar(
    theme=RichProgressBarTheme(
        description="green_yellow",
        progress_bar="green1",
        progress_bar_finished="green1",
        progress_bar_pulse="#6206E0",
        batch_progress="green_yellow",
        time="grey82",
        processing_speed="grey82",
        metrics="grey82",
    )
)

In [37]:
trainer = pl.Trainer(max_epochs = 20, accelerator = "auto", enable_progress_bar = True, strategy = "auto", callbacks=progress_bar)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(contrastive_model, sscl_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name      ┃ Type              ┃ Params ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ vit       │ ViT               │  4.6 M │
│ 1 │ criterion │ TripletMarginLoss │      0 │
└───┴───────────┴───────────────────┴────────┘

Trainable params: 4.6 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.6 M                                                                                                
Total estimated model params size (MB): 18

/home/201851018/.conda/envs/pytorchenv/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/201851018/.conda/envs/pytorchenv/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None

Output()

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch 1/19 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━ 4874/6250 1:21:31 • 0:17:51 1.29it/s v_num: 30 train_loss: 0.0 

In [51]:
class ClassificationDataset(Dataset):

    def __init__(self, dataset):
        # Transforms
        self.dataset = dataset
        self.to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize(size = (128, 128))
        self.grayscale = transforms.Grayscale()
        self.root_dir = "images/"
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        
        row = self.dataset.iloc[idx]
        image = self.resize(self.to_tensor(Image.open(self.root_dir + row['img_path'])))
#         return self.grayscale(image), torch.Tensor([row['has_under_extrusion']])
        return image, torch.Tensor([row['has_under_extrusion']])    

In [52]:
classification_dataloader = DataLoader(
    ClassificationDataset(train_df),
    batch_size=16,
    shuffle=True,
    pin_memory=True,
    num_workers=2
)

In [8]:
class ExtrusionClassifier(pl.LightningModule):
    
    def __init__(self, contrastive_model):
        
        super().__init__()
        self.contrastive_model = contrastive_model
        self.mlp = nn.Sequential(
            nn.Linear(1000, 256),
            nn.PReLU(),
            nn.Linear(256, 1)
        )
        self.criterion = nn.BCEWithLogitsLoss()
        
    def forward(self, x):
        return self.mlp(self.contrastive_model(x))
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam([
                {'params': self.contrastive_model.parameters(), 'lr': 1e-5},
                {'params': self.mlp.parameters(), 'lr': 1e-3}
            ])
#         scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 5)
        
        return [optimizer]
    
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        
        loss = self.criterion(self(x), y)
        self.log("train_loss", loss.item(), prog_bar=True)
        return loss
        

In [9]:
classifier = ExtrusionClassifier(contrastive_model)


In [10]:
progress_bar = RichProgressBar(
    theme=RichProgressBarTheme(
        description="green_yellow",
        progress_bar="green1",
        progress_bar_finished="green1",
        progress_bar_pulse="#6206E0",
        batch_progress="green_yellow",
        time="grey82",
        processing_speed="grey82",
        metrics="grey82",
    )
)


In [11]:
trainer_classifier = pl.Trainer(max_epochs = 10, accelerator = "auto", enable_progress_bar = True, strategy = "auto", callbacks=progress_bar)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [57]:
trainer_classifier.fit(classifier, classification_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


┏━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name              ┃ Type              ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ contrastive_model │ ContrastiveModel  │  2.6 M │
│ 1 │ mlp               │ Sequential        │  256 K │
│ 2 │ criterion         │ BCEWithLogitsLoss │      0 │
└───┴───────────────────┴───────────────────┴────────┘

Trainable params: 2.8 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 2.8 M                                                                                                
Total estimated model params size (MB): 11

/home/201851018/.conda/envs/pytorchenv/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/201851018/.conda/envs/pytorchenv/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None

Output()

/home/201851018/.conda/envs/pytorchenv/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 72 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate

In [79]:
torch.save(classifier.state_dict(), open("classifer.pth", "wb"))

In [12]:
classifier.load_state_dict(torch.load(open("classifer.pth", "rb")))

<All keys matched successfully>

In [13]:
class ClassificationDatasetTest(Dataset):

    def __init__(self, dataset):
        # Transforms
        self.dataset = dataset
        self.to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize(size = (128, 128))
        self.grayscale = transforms.Grayscale()
        self.root_dir = "images/"
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        
        row = self.dataset.iloc[idx]
        image = self.resize(self.to_tensor(Image.open(self.root_dir + row['img_path'])))
        return image

In [14]:
test_df = pd.read_csv("test.csv")
test_dataset = ClassificationDatasetTest(test_df)

In [15]:
classification_dataloader = DataLoader(
    ClassificationDatasetTest(test_df),
    batch_size=16,
#     shuffle=True,
    pin_memory=True,
    num_workers=2
)

In [16]:
preds = trainer_classifier.predict(classifier, classification_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]
/home/201851018/.conda/envs/pytorchenv/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/201851018/.conda/envs/pytorchenv/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which

Output()

/home/201851018/.conda/envs/pytorchenv/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/201851018/.conda/envs/pytorchenv/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None

In [22]:
(torch.sigmoid(torch.cat(preds)) > 0.5).numpy().astype('int')

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [21]:
inp = []
pred = []
for data in tqdm(classification_dataloader, total = len(test_dataset)):
    
    x = data.cuda()
    
    y_hat = torch.sigmoid(classifier(x))[0].item() > 0.5
    
    pred.append(int(y_hat))

  0%|          | 48/25279 [00:11<1:42:41,  4.10it/s]


KeyboardInterrupt: 

In [31]:
# test_df['has_under_extrusion'] = (torch.sigmoid(torch.cat(preds)) > 0.5).numpy().astype('int')
test_df['has_under_extrusion'] = (x/5 > 0.5).astype("int")

In [32]:
test_df[['img_path', 'has_under_extrusion']].to_csv("submission6.csv", index = False)

In [29]:
x = np.zeros(25279, )
for i in range(1, 6):
    x+=pd.read_csv(f"submission{i}.csv")['has_under_extrusion']

0         True
1         True
2         True
3         True
4         True
         ...  
25274     True
25275     True
25276     True
25277    False
25278     True
Name: has_under_extrusion, Length: 25279, dtype: bool